In [3]:
import pandas as pd
import json
import os

In [10]:
lang_files = {}
for file in os.listdir('../data/train'):
    try:
        path = f'../data/train/{file}'
        jsonObj = pd.read_json(path_or_buf=f'../data/train/{file}/train.jsonl', lines=True)
        lang_files[file.split(".")[0]] = jsonObj
    except Exception as e:
        print(file, e)
    


lang_files['es'].columns



Index(['id', 'source_locale', 'target_locale', 'source', 'target', 'entities',
       'from'],
      dtype='object')

Accepts a question id, a list of languages and returns if that ID is present in all

In [32]:
multilingual_df = pd.DataFrame(columns=['q_id', 'en', 'ar', 'de', 'es', 'fr', 'it', 'ja'])
langs = ['ar', 'de', 'es', 'fr', 'it', 'ja']
ids_in_all = []

def id_in_all(question_id:str, languages:list=langs, exists_in_all:bool=True, write:bool=False):
    for lang in languages:
        if question_id not in lang_files[lang].id.values:
            exists_in_all = False

        if write==True:


            for lang in languages:
                

for id in lang_files['es'].id.values:
    if id_in_all(id):
        row = 
        new_row = {'id': 4, 'source': 'D'}



print(ids_in_all)
print(len(ids_in_all))

['f477742c', '650e81a3', '33ed28dd', '9508d042', '47feccd4', '4c5901ae', '375001ec', '3263c8a7', '7175a825', '0d9c0aba', '58706029', '4de034f6', '4df2564a', '5a7a384e', '5d65d703', 'fa08c557', '66e66d5d', '9bcc144a', '4001d7e5', '1cbe8494', '15a76dda', '93ce505b', '87b569b1', 'bad12250', '96a9bd38', 'e74425b1', '35575c6b', 'ecb7180c', '11e50432', '7f0edb54', '89b08063', 'a8927bb4', '0c5f2e78', '7d8a94d2', 'c2dd55fb', '76279f72', '930d8d06', '28f1de76', 'af75458f', '7cefb323', 'b8c86234', '4d2011f2', 'b739b77d', '61c45307', '00abdc69', '688d1821', '8f516b49', '9342d585', '74d115a2', 'b89144bf', 'cf0252e7', 'cef01d78', 'f50b5bd7', 'e0510724', '9e966d9e', '616f39aa', 'a82173d6', '624759b4', '66aea708', '2c202d72', '75af68f0', 'da180721', '85588ff9', '91639013', '7f2ffe2e', '2af2f393', '1bee86b8', 'c8bc3cee', 'efa5d49c', '6ec5541b', '3188dad1', '9cf856ba', '0314da0f', '3322edb3', '75474258', '8cd0f39b', 'c0a6a234', '67efb0d5', '995ab6eb', '5be7ad45', '7c46063e', '29166b5a', 'e6862885', '62